# Expirence 4

In [101]:
import math
import numpy as np

# Analysis and plotting modules
import pandas as pd
import plotly
import plotly.express as px

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD import configs

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor


pd.options.plotting.backend = "plotly"

## 3.a

In [102]:
df = pd.read_csv('data/teste_3600-ciclos-20.txt', delimiter = "\t", header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,0,0,0,0,0,0,0,25.433,0,...,0,0,0,0,0,0,0,0,0,3600
1,0,0,0,0,0,5,0,0,25.482,0,...,0,0,0,0,0,0,0,0,0,3600
2,0,0,0,0,0,10,0,0,25.427,0,...,0,0,0,0,0,0,0,0,0,3600
3,0,0,0,0,0,15,0,0,25.807,0,...,0,0,0,0,0,0,0,0,0,3600
4,0,0,0,0,0,20,0,0,25.994,0,...,0,0,0,0,0,0,0,0,0,3600


In [103]:
cols_dict = {
    5:'time (s)',
    8:'Tc',
    19:'Ti',
    24:'Te',
    31:'Ta',
    41:'Freq'
}

df.rename(columns=cols_dict, inplace=True)
df = df[cols_dict.values()]
df[['Ta', 'Ti', 'Te', 'Tc']] = df[['Ta', 'Ti', 'Te', 'Tc']]+273.15
df['S'] = df['Freq'].apply(lambda x: 1 if x>0 else 0)
df['Freq'] = df['Freq']/30
df

,time (s),Tc,Ti,Te,Ta,Freq,S
0,0,298.583,255.215,261.078,293.009,120.0,1
1,5,298.632,255.259,260.997,292.990,120.0,1
2,10,298.577,255.269,260.707,293.039,120.0,1
3,15,298.957,255.302,259.572,292.973,120.0,1
4,20,299.144,255.308,259.003,293.011,120.0,1
...,...,...,...,...,...,...,...
1016,5080,298.353,255.106,261.118,293.015,0.0,0
1017,5085,298.402,255.122,261.158,292.992,0.0,0
1018,5090,298.364,255.146,261.199,292.990,0.0,0
1019,5095,298.321,255.208,261.118,293.007,0.0,0


In [104]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(y=df['S'], x=df['time (s)'], name="S"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(y=df['Freq'], x=df['time (s)'], name="Frequencia"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Frequencia e Chaveamento no tempo"
)

# Set x-axis title
fig.update_xaxes(title_text="tempo (s)")

# Set y-axes titles
fig.update_yaxes(title_text="S (absoluto)", secondary_y=False)
fig.update_yaxes(title_text="Frequencia (Hz)", secondary_y=True)

fig.show()

In [105]:
fig = px.line(df, x='time (s)', y='Freq')
fig.show()

In [106]:
fig = px.line(df, x='time (s)', y=['Ta', 'Ti', 'Te', 'Tc'])
fig.update_yaxes(title_text="Temperatura (K)")
fig.update_xaxes(title_text="Tempo (s)")
fig.update_layout(
    title_text="Temperaturas medidas no tempo"
)

fig.show()

## 3.b

In [107]:
df

,time (s),Tc,Ti,Te,Ta,Freq,S
0,0,298.583,255.215,261.078,293.009,120.0,1
1,5,298.632,255.259,260.997,292.990,120.0,1
2,10,298.577,255.269,260.707,293.039,120.0,1
3,15,298.957,255.302,259.572,292.973,120.0,1
4,20,299.144,255.308,259.003,293.011,120.0,1
...,...,...,...,...,...,...,...
1016,5080,298.353,255.106,261.118,293.015,0.0,0
1017,5085,298.402,255.122,261.158,292.992,0.0,0
1018,5090,298.364,255.146,261.199,292.990,0.0,0
1019,5095,298.321,255.208,261.118,293.007,0.0,0


In [108]:
X = df[['Tc', 'Ti', 'Te', 'Ta', 'S', 'Freq']][:-1]
X

,Tc,Ti,Te,Ta,S,Freq
0,298.583,255.215,261.078,293.009,1,120.0
1,298.632,255.259,260.997,292.990,1,120.0
2,298.577,255.269,260.707,293.039,1,120.0
3,298.957,255.302,259.572,292.973,1,120.0
4,299.144,255.308,259.003,293.011,1,120.0
...,...,...,...,...,...,...
1015,298.370,255.053,261.118,293.043,0,0.0
1016,298.353,255.106,261.118,293.015,0,0.0
1017,298.402,255.122,261.158,292.992,0,0.0
1018,298.364,255.146,261.199,292.990,0,0.0


In [109]:
Y = df[['Tc', 'Ti', 'Te', 'time (s)']].diff()[1:]
Y

,Tc,Ti,Te,time (s)
1,0.049,0.044,-0.081,5.0
2,-0.055,0.010,-0.290,5.0
3,0.380,0.033,-1.135,5.0
4,0.187,0.006,-0.569,5.0
5,0.118,0.053,-0.599,5.0
...,...,...,...,...
1016,-0.017,0.053,0.000,5.0
1017,0.049,0.016,0.040,5.0
1018,-0.038,0.024,0.041,5.0
1019,-0.043,0.062,-0.081,5.0


In [110]:
def MMQ(X, y):
    XT = np.transpose(X)
    temp1 = np.linalg.inv(np.matmul(XT, X)) #(XTX)^-1
    temp2 = np.matmul(temp1, XT) #(XTX)^-1XT
    P = np.matmul(temp2, y)
    return P

In [111]:
Xti = np.array(X[['Ti','Ta','Te']])
yti = np.array(Y['Ti']/Y['time (s)'])

Pti = MMQ(Xti, yti)
Pti

array([-2.03382044e-05, -5.80117985e-04,  6.93337716e-04])

In [112]:
X_te = X.copy()
X_te = X_te[['Ti']]
X_te['STc'] = X['Tc']*X['S']
X_te['Te'] = X['Te']
X_te['STe'] = X['Te']*X['S']

Xte = np.array(X_te[['Ti','STc','Te','STe']])
yte = np.array(Y['Te']/Y['time (s)'])

Pte = MMQ(Xte, yte)
Pte

array([ 0.00290309,  0.00169553, -0.00280383, -0.00238564])

In [113]:
X_tc = pd.DataFrame()
X_tc['STe'] = X['Te']*X['S']
X_tc['STc'] = X['Tc']*X['S']
X_tc['Sw'] = X['S']*X['Freq']

Xtc = np.array(X_tc[['STe', 'STc', 'Sw']])
ytc = np.array(Y['Tc']/Y['time (s)'])

Ptc = MMQ(Xtc, ytc)
Ptc

array([-0.00328081, -0.00794766,  0.02772387])

In [114]:
fi_array = []
coefs = [Pti, Pte, Ptc]
for l in coefs:
    for c in l:
        fi_array.append(c)

fi_array

[-2.0338204426124768e-05,
 -0.0005801179849686812,
 0.0006933377156641724,
 0.0029030872083009376,
 0.001695529645951039,
 -0.0028038347772006453,
 -0.0023856379589366787,
 -0.0032808110200568627,
 -0.007947662349708246,
 0.027723871983645152]

# 4 (revisar)

In [115]:
from scipy.optimize import fsolve
import math

def equations(p):
    Ci, Rai, Rei, Ce, Rec, Cc, Rep, gama, Rcp = p
    eq1 = -fi_array[0] - (1/(Ci*Rai)) - (1/(Ci*Rei))
    eq2 = (1/(Ci*Rai)) - fi_array[1]
    eq3 = (1/(Ci*Rei)) - fi_array[2]
    eq4 = (1/Ce*Rei)  - fi_array[3]
    eq5 = - (1/Ce*Rec)  - fi_array[4]
    eq6 = - (1/Ce*Rei)  - fi_array[5]
    eq7 = - (1/Ce*Rec)  - fi_array[6]
    eq8 = - (1/Cc*Rcp) - (1/Cc*Rec)  - fi_array[7]
#    eq9 = - (1/Cc*Rec)  - fi_array[8]
    eq10 = (gama/Cc*Rcp)  - fi_array[9]
    
    return (eq1, eq2, eq3, eq4, eq5,
            eq6, eq7, eq8, eq10)

parameteres =  fsolve(equations, (1, 1, 1, 1,
                                  1, 1, 1, 1, 1))
parameteres

/home/yves/.cache/pypoetry/virtualenvs/process-simulation-6FL6Grl8-py3.7/lib/python3.7/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning:

The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.



array([ 8.19401342e+00,  6.12408631e+00,  1.05765107e+00,  9.97361005e+00,
        2.29309783e-02,  8.98048339e+00,  4.29263462e+02, -7.81173371e-01,
       -2.15998719e-01])

In [116]:
- (1/parameteres[5]*parameteres[6])  - fi_array[8]

-47.7916465684587

Usando todas as equações, o resultado n bate. Logo o sistema n tem solução

# 5

In [117]:
initial_state = df.rename(columns={'time (s)': 'time'}).iloc[0,:].to_dict()

initial_state

{'time': 0.0,
 'Tc': 298.58299999999997,
 'Ti': 255.21499999999997,
 'Te': 261.078,
 'Ta': 293.00899999999996,
 'Freq': 120.0,
 'S': 1.0}

In [118]:
system_params = {}
i = 0
for p in fi_array:
    system_params['p'+str(i)] = [p]
    i+=1
system_params['timestep'] = [5]
system_params

{'p0': [-2.0338204426124768e-05],
 'p1': [-0.0005801179849686812],
 'p2': [0.0006933377156641724],
 'p3': [0.0029030872083009376],
 'p4': [0.001695529645951039],
 'p5': [-0.0028038347772006453],
 'p6': [-0.0023856379589366787],
 'p7': [-0.0032808110200568627],
 'p8': [-0.007947662349708246],
 'p9': [0.027723871983645152],
 'timestep': [5]}

In [119]:
def s_ti(params, substep, state_history, previous_state, policy_input):
    a = previous_state['Ti']*params['p0']
    b = previous_state['Ta']*params['p1']
    c = previous_state['Te']*params['p2']
    return 'Ti', ((a+b+c)*params['timestep']) + previous_state['Ti']

def s_te(params, substep, state_history, previous_state, policy_input):
    a = previous_state['Ti']*params['p3']
    b = previous_state['S']*previous_state['Tc']*params['p4']
    c = previous_state['Te']*params['p5']
    d = previous_state['S']*previous_state['Te']*params['p6']
    return 'Te', ((a+b+c+d)*params['timestep']) + previous_state['Te']

def s_tc(params, substep, state_history, previous_state, policy_input):
    a = previous_state['S']*previous_state['Te']*params['p7']
    b = previous_state['S']*previous_state['Tc']*params['p8']
    c = previous_state['S']*previous_state['Freq']*params['p9']
    return 'Tc', ((a+b+c)*params['timestep']) + previous_state['Tc']

def s_freq(params, substep, state_history, previous_state, policy_input):
    i = (previous_state['time']/5) + 1
    return 'Freq', df.loc[i,'Freq']

def s_S(params, substep, state_history, previous_state, policy_input):
    i = (previous_state['time']/5) + 1
    return 'S', df.loc[i,'S']

def s_ta(params, substep, state_history, previous_state, policy_input):
    i = (previous_state['time']/5) + 1
    return 'Ta', df.loc[i,'Ta']

def s_time(params, substep, state_history, previous_state, policy_input):
    return 'time', previous_state['time']+params['timestep']


In [120]:
partial_state_update_blocks = [
    {
        'policies': {},
        'variables': {
            'time': s_time,
            'S': s_S,
            'Freq': s_freq,
            'Ta': s_ta,
            'Tc': s_tc,
            'Te': s_te,
            'Ti': s_ti
        }
    }
]

In [121]:
sim_config = config_sim({
    "N": 1,
    "T": range(1020),
    "M": system_params
})

del configs[:] # Clear any prior configs
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)
configs[-1].__dict__

{'sim_config': {'N': 1,
  'T': range(0, 1020),
  'M': {'p0': -2.0338204426124768e-05,
   'p1': -0.0005801179849686812,
   'p2': 0.0006933377156641724,
   'p3': 0.0029030872083009376,
   'p4': 0.001695529645951039,
   'p5': -0.0028038347772006453,
   'p6': -0.0023856379589366787,
   'p7': -0.0032808110200568627,
   'p8': -0.007947662349708246,
   'p9': 0.027723871983645152,
   'timestep': 5},
  'subset_id': 0,
  'subset_window': deque([0, None]),
  'simulation_id': 0,
  'run_id': 0},
 'initial_state': {'time': 0.0,
  'Tc': 298.58299999999997,
  'Ti': 255.21499999999997,
  'Te': 261.078,
  'Ta': 293.00899999999996,
  'Freq': 120.0,
  'S': 1.0},
 'seeds': {},
 'env_processes': {},
 'exogenous_states': {},
 'partial_state_updates': [{'policies': {},
   'variables': {'time': <function __main__.s_time(params, substep, state_history, previous_state, policy_input)>,
    'S': <function __main__.s_S(params, substep, state_history, previous_state, policy_input)>,
    'Freq': <function __main__.s_

In [122]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (1020, 11, 1, 7)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.20s


In [123]:
simulation_result = pd.DataFrame(raw_result)
simulation_result

,time,Tc,Ti,Te,Ta,Freq,S,simulation,subset,run,substep,timestep
0,0.0,298.583000,255.215000,261.078000,293.009,120.0,1.0,0,0,1,0,0
1,5.0,299.069401,255.244224,260.539553,292.990,120.0,1.0,0,0,1,1,1
2,10.0,299.545306,255.271634,260.019625,293.039,120.0,1.0,0,0,1,1,2
3,15.0,300.010828,255.297096,259.517620,292.973,120.0,1.0,0,0,1,1,3
4,20.0,300.466086,255.321007,259.032957,293.011,120.0,1.0,0,0,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
1016,5080.0,317.635994,252.558241,259.796483,293.015,0.0,0.0,0,0,1,1,1016
1017,5085.0,317.635994,252.583275,259.820344,292.992,0.0,0.0,0,0,1,1,1017
1018,5090.0,317.635994,252.608456,259.844234,292.990,0.0,0.0,0,0,1,1,1018
1019,5095.0,317.635994,252.633723,259.868155,293.007,0.0,0.0,0,0,1,1,1019


In [124]:
simulation_result['Te_simulado'] = simulation_result['Te']
simulation_result['Ti_simulado'] = simulation_result['Ti']
simulation_result['Tc_simulado'] = simulation_result['Tc']

simulation_result['Ti_real'] = df['Ti']
simulation_result['Te_real'] = df['Te']
simulation_result['Tc_real'] = df['Tc']

In [125]:
fig = px.line(simulation_result, x='time', y=['Te_simulado', 'Te_real'])
fig.update_yaxes(title_text="Temperatura (K)")
fig.update_xaxes(title_text="Tempo (s)")
fig.update_layout(
    title_text="Simulação do Evaporador no tempo"
)
fig.show()

In [126]:
fig = px.line(simulation_result, x='time', y=['Ti_simulado', 'Ti_real'])
fig.update_yaxes(title_text="Temperatura (K)")
fig.update_xaxes(title_text="Tempo (s)")
fig.update_layout(
    title_text="Simulação do gabinete no tempo"
)
fig.show()

In [127]:
fig = px.line(simulation_result, x='time', y=['Tc_simulado', 'Tc_real'])
fig.update_yaxes(title_text="Temperatura (K)")
fig.update_xaxes(title_text="Tempo (s)")
fig.update_layout(
    title_text="Simulação do Condensador no tempo"
)
fig.show()

In [38]:
def mse(y1, y2):
    value = 0
    n = len(y1)
    for i in range(n):
        value+=((y1[i]-y2[i])**2)
    return value/n

def mape(y1, y2):
    value = 0
    n = len(y1)
    for i in range(n):
        value+=(((y1[i]-y2[i])**2)/y1[i])
    return value/n

def R2(y1, y2):
    value1 = 0
    value2 = 0
    mean_y1 = y1.mean()
    n = len(y1)
    for i in range(n):
        value1+=((y1[i]-y2[i])**2)
        value2+=((y1[i]-mean_y1)**2)
    return 1-(value1/value2)

In [128]:
# Te

print(mse(simulation_result['Te_real'], simulation_result['Te']))
print(mape(simulation_result['Te_real'], simulation_result['Te']))
print(R2(simulation_result['Te_real'], simulation_result['Te']))

1.844376032637538
0.0071864350127075075
0.9521347892869958


In [129]:
# Ti

print(mse(simulation_result['Ti_real'], simulation_result['Ti']))
print(mape(simulation_result['Ti_real'], simulation_result['Ti']))
print(R2(simulation_result['Ti_real'], simulation_result['Ti']))

1.6925384673299104
0.0066786100371547434
0.4714997038666534


In [130]:
# Tc

print(mse(simulation_result['Tc_real'], simulation_result['Tc']))
print(mape(simulation_result['Tc_real'], simulation_result['Tc']))
print(R2(simulation_result['Tc_real'], simulation_result['Tc']))

144.46167685421744
0.4814847755216067
-1.4632197382346788


# 6.a

\begin{equation}
T_i [k+1] = T_i [k] + \cfrac{\tau}{C_i} \left[\cfrac{T_a [k] - Ti [k]}{R{ai}} - \cfrac{T_i [k] - Te [k]}{R{ei}} + 11.5S[k]\right] 
\end{equation}

# 6.b

\begin{equation}
T_c [k+1] = T_c [k] + \cfrac{\tau S[k]}{C_c} \left[\cfrac{T_p [k] - Tc [k]}{R{cp}} - \cfrac{T_c [k] + Te [k]}{R{ec}} - (1-S[k])\cfrac{Tc-Ta}{Rca} \right]
\end{equation}

# 7.a

In [146]:
X_ti = X[['Ti','Ta','Te']].copy()
X_ti['11.5S'] = X['S']*11.5
Xti = np.array(X_ti)
yti = np.array(Y['Ti']/Y['time (s)'])

Pti = MMQ(Xti, yti)
Pti

array([ 6.79896769e-05, -6.25968675e-04,  6.59057548e-04, -5.06476561e-05])

In [147]:
X_te = X.copy()
X_te = X_te[['Ti']]
X_te['STc'] = X['Tc']*X['S']
X_te['Te'] = X['Te']
X_te['STe'] = X['Te']*X['S']

Xte = np.array(X_te[['Ti','STc','Te','STe']])
yte = np.array(Y['Te']/Y['time (s)'])

Pte = MMQ(Xte, yte)
Pte

array([ 0.00290309,  0.00169553, -0.00280383, -0.00238564])

In [148]:
X_tc = pd.DataFrame()
X_tc['STe'] = X['Te']*X['S']
X_tc['STc'] = X['Tc']*X['S']
X_tc['Sw'] = X['S']*X['Freq']
X_tc['(1-S)Tc'] = (X['S']-1)*X['Tc']
X_tc['(1-S)Ta'] = (1-X['S'])*X['Ta']

Xtc = np.array(X_tc)
ytc = np.array(Y['Tc']/Y['time (s)'])

Ptc = MMQ(Xtc, ytc)
Ptc

array([-0.00328081, -0.00794766,  0.02772387,  0.00328255,  0.00333921])

In [149]:
fi_array = []
coefs = [Pti, Pte, Ptc]
for l in coefs:
    for c in l:
        fi_array.append(c)

fi_array

[6.798967691282256e-05,
 -0.0006259686747535028,
 0.0006590575477011156,
 -5.0647656101802606e-05,
 0.0029030872083009376,
 0.001695529645951039,
 -0.0028038347772006453,
 -0.0023856379589366787,
 -0.0032808110200568614,
 -0.007947662349708248,
 0.027723871983645152,
 0.0032825465545404486,
 0.003339213986540125]

# 7.b

In [150]:
initial_state = df.rename(columns={'time (s)': 'time'}).iloc[0,:].to_dict()

initial_state

{'time': 0.0,
 'Tc': 298.58299999999997,
 'Ti': 255.21499999999997,
 'Te': 261.078,
 'Ta': 293.00899999999996,
 'Freq': 120.0,
 'S': 1.0}

In [136]:
system_params = {}
i = 0
for p in fi_array:
    system_params['p'+str(i)] = [p]
    i+=1
system_params['timestep'] = [5]
system_params

{'p0': [6.798967691282256e-05],
 'p1': [-0.0006259686747535028],
 'p2': [0.0006590575477011156],
 'p3': [-5.0647656101802606e-05],
 'p4': [0.0029030872083009376],
 'p5': [0.001695529645951039],
 'p6': [-0.0028038347772006453],
 'p7': [-0.0023856379589366787],
 'p8': [-0.0032808110200568614],
 'p9': [-0.007947662349708248],
 'p10': [0.027723871983645152],
 'p11': [0.0032825465545404486],
 'p12': [0.003339213986540125],
 'timestep': [5]}

In [137]:
def s_ti(params, substep, state_history, previous_state, policy_input):
    a = previous_state['Ti']*params['p0']
    b = previous_state['Ta']*params['p1']
    c = previous_state['Te']*params['p2']
    d = previous_state['S']*params['p3']*11.5
    return 'Ti', ((a+b+c+d)*params['timestep']) + previous_state['Ti']

def s_te(params, substep, state_history, previous_state, policy_input):
    a = previous_state['Ti']*params['p4']
    b = previous_state['S']*previous_state['Tc']*params['p5']
    c = previous_state['Te']*params['p6']
    d = previous_state['S']*previous_state['Te']*params['p7']
    return 'Te', ((a+b+c+d)*params['timestep']) + previous_state['Te']

def s_tc(params, substep, state_history, previous_state, policy_input):
    a = previous_state['S']*previous_state['Te']*params['p8']
    b = previous_state['S']*previous_state['Tc']*params['p9']
    c = previous_state['S']*previous_state['Freq']*params['p10']
    d = (previous_state['S']-1)*previous_state['Tc']*params['p11']
    e = (1-previous_state['S'])*previous_state['Ta']*params['p12']
    return 'Tc', ((a+b+c+d+e)*params['timestep']) + previous_state['Tc']

def s_freq(params, substep, state_history, previous_state, policy_input):
    i = (previous_state['time']/5) + 1
    return 'Freq', df.loc[i,'Freq']

def s_S(params, substep, state_history, previous_state, policy_input):
    i = (previous_state['time']/5) + 1
    return 'S', df.loc[i,'S']

def s_ta(params, substep, state_history, previous_state, policy_input):
    i = (previous_state['time']/5) + 1
    return 'Ta', df.loc[i,'Ta']

def s_time(params, substep, state_history, previous_state, policy_input):
    return 'time', previous_state['time']+params['timestep']


In [138]:
partial_state_update_blocks = [
    {
        'policies': {},
        'variables': {
            'time': s_time,
            'S': s_S,
            'Freq': s_freq,
            'Ta': s_ta,
            'Tc': s_tc,
            'Te': s_te,
            'Ti': s_ti
        }
    }
]

In [139]:
sim_config = config_sim({
    "N": 1,
    "T": range(1020),
    "M": system_params
})

del configs[:] # Clear any prior configs
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)
configs[-1].__dict__

{'sim_config': {'N': 1,
  'T': range(0, 1020),
  'M': {'p0': 6.798967691282256e-05,
   'p1': -0.0006259686747535028,
   'p2': 0.0006590575477011156,
   'p3': -5.0647656101802606e-05,
   'p4': 0.0029030872083009376,
   'p5': 0.001695529645951039,
   'p6': -0.0028038347772006453,
   'p7': -0.0023856379589366787,
   'p8': -0.0032808110200568614,
   'p9': -0.007947662349708248,
   'p10': 0.027723871983645152,
   'p11': 0.0032825465545404486,
   'p12': 0.003339213986540125,
   'timestep': 5},
  'subset_id': 0,
  'subset_window': deque([0, None]),
  'simulation_id': 0,
  'run_id': 0},
 'initial_state': {'time': 0.0,
  'Tc': 298.58299999999997,
  'Ti': 255.21499999999997,
  'Te': 261.078,
  'Ta': 293.00899999999996,
  'Freq': 120.0,
  'S': 1.0},
 'seeds': {},
 'env_processes': {},
 'exogenous_states': {},
 'partial_state_updates': [{'policies': {},
   'variables': {'time': <function __main__.s_time(params, substep, state_history, previous_state, policy_input)>,
    'S': <function __main__.s_S

In [140]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (1020, 14, 1, 7)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.20s


In [141]:
simulation_result = pd.DataFrame(raw_result)
simulation_result

,time,Tc,Ti,Te,Ta,Freq,S,simulation,subset,run,substep,timestep
0,0.0,298.583000,255.215000,261.078000,293.009,120.0,1.0,0,0,1,0,0
1,5.0,299.069401,255.242103,260.539553,292.990,120.0,1.0,0,0,1,1,1
2,10.0,299.545306,255.267499,260.019594,293.039,120.0,1.0,0,0,1,1,2
3,15.0,300.010829,255.291038,259.517530,292.973,120.0,1.0,0,0,1,1,3
4,20.0,300.466088,255.313137,259.032781,293.011,120.0,1.0,0,0,1,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...
1016,5080.0,298.282530,249.785616,257.475275,293.015,0.0,0.0,0,0,1,1,1016
1017,5085.0,298.279098,249.801894,257.491431,292.992,0.0,0.0,0,0,1,1,1017
1018,5090.0,298.275337,249.818303,257.507597,292.990,0.0,0.0,0,0,1,1,1018
1019,5095.0,298.271606,249.834778,257.523775,293.007,0.0,0.0,0,0,1,1,1019


In [142]:
simulation_result['Te_simulado'] = simulation_result['Te']
simulation_result['Ti_simulado'] = simulation_result['Ti']
simulation_result['Tc_simulado'] = simulation_result['Tc']

simulation_result['Ti_real'] = df['Ti']
simulation_result['Te_real'] = df['Te']
simulation_result['Tc_real'] = df['Tc']

In [143]:
fig = px.line(simulation_result, x='time', y=['Te_simulado', 'Te_real'])
fig.update_yaxes(title_text="Temperatura (K)")
fig.update_xaxes(title_text="Tempo (s)")
fig.update_layout(
    title_text="Simulação do Evaporador no tempo"
)
fig.show()

In [144]:
fig = px.line(simulation_result, x='time', y=['Ti_simulado', 'Ti_real'])
fig.update_yaxes(title_text="Temperatura (K)")
fig.update_xaxes(title_text="Tempo (s)")
fig.update_layout(
    title_text="Simulação do gabinete no tempo"
)
fig.show()

In [145]:
fig = px.line(simulation_result, x='time', y=['Tc_simulado', 'Tc_real'])
fig.update_yaxes(title_text="Temperatura (K)")
fig.update_xaxes(title_text="Tempo (s)")
fig.update_layout(
    title_text="Simulação do Condensador no tempo"
)
fig.show()

In [89]:
# Te

print(mse(simulation_result['Te_real'], simulation_result['Te']))
print(mape(simulation_result['Te_real'], simulation_result['Te']))
print(R2(simulation_result['Te_real'], simulation_result['Te']))

4.123507391331028
0.015992033598917262
0.8929868168583612


In [90]:
# Ti

print(mse(simulation_result['Ti_real'], simulation_result['Ti']))
print(mape(simulation_result['Ti_real'], simulation_result['Ti']))
print(R2(simulation_result['Ti_real'], simulation_result['Ti']))

5.422751927019357
0.021416087785529083
-0.6932708204904736


In [91]:
# Tc
print(mse(simulation_result['Tc_real'], simulation_result['Tc']))
print(mape(simulation_result['Tc_real'], simulation_result['Tc']))
print(R2(simulation_result['Tc_real'], simulation_result['Tc']))

0.4774186560666171
0.0015360733599634902
0.9918595223132326


# 8

In [ ]:
Re, Ra